In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import torch

In [ ]:
# Load GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained('gpt2') #get the GPT 2 model

In [ ]:
# Load dataset (you can use your dataset instead)
dataset = load_dataset('emotion')

In [ ]:
# Prepare the dataset with labels for bias (1 for bias, 0 for non-bias)
train_texts = [example['text'] for example in dataset['train']]
train_labels = [1 if 'bias_keyword' in text.lower() else 0 for text in train_texts]  # Example labels, replace with real conditions

In [ ]:
# Tokenize the data and add a classification label
# Adding [BIAS] or [NONBIAS] label to the end of each text for classification purposes
def tokenize_function(examples):
    texts = []
    for text, label in zip(examples['text'], examples['label']):
        if label == 1:
            texts.append(f"{text} [BIAS]")
        else:
            texts.append(f"{text} [NONBIAS]")
    # Tokenizing texts with padding and truncation for uniform input size
    return tokenizer(texts, padding='max_length', truncation=True, pad_to_max_length=True)

In [ ]:
# Create tokenized dataset
tokenized_dataset = Dataset.from_dict({'text': train_texts, 'label': train_labels})

In [ ]:
tokenized_dataset = tokenized_dataset.map(tokenize_function, batched=True)

In [ ]:
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

In [ ]:
# Training arguments
# Setting up training parameters such as batch size, number of epochs, logging frequency, etc.
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    save_steps=500,
    logging_dir="./logs",
    logging_steps=10,
)

# # Create Trainer instance
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_dataset,
# )

# Create Trainer instance
# Initializing the Trainer class with the model, training arguments, dataset, and tokenizer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=lambda data: {'input_ids': torch.stack([f['input_ids'] for f in data]),
                                'attention_mask': torch.stack([f['attention_mask'] for f in data]),
                                'labels': torch.stack([f['label'] for f in data]).view(-1)}
    # data_collator=lambda data: {'input_ids': torch.stack([f['input_ids'] for f in data]),
    #                             'attention_mask': torch.stack([f['attention_mask'] for f in data]),
    #                             'labels': torch.stack([f['label'].unsqueeze(0) for f in data])}

    # data_collator=lambda data: {'input_ids': torch.stack([f['input_ids'] for f in data]),
    #                         'attention_mask': torch.stack([f['attention_mask'] for f in data]),
    #                         'labels': torch.tensor([f['label'] for f in data])}
    
    # data_collator=lambda data: {'input_ids': torch.stack([f['input_ids'] for f in data]),
    #                         'attention_mask': torch.stack([f['attention_mask'] for f in data]),
    #                         'labels': torch.stack([torch.tensor(f['label']) for f in data])}

    # data_collator=lambda data: {'input_ids': torch.nn.utils.rnn.pad_sequence([f['input_ids'] for f in data], batch_first=True),
    #                             'attention_mask': torch.nn.utils.rnn.pad_sequence([f['attention_mask'] for f in data], batch_first=True),
    #                             'labels': torch.tensor([f['label'] for f in data], dtype=torch.long)}

)

In [ ]:
# Train the model
trainer.train()

In [ ]:
# Use the fine-tuned model to classify new texts
def classify_bias(text):
    inputs = tokenizer.encode(text, return_tensors="pt")
    output = model.generate(inputs, max_length=len(inputs[0]) + 10)
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
    
    if "[BIAS]" in decoded_output:
        return "Bias detected"
    elif "[NONBIAS]" in decoded_output:
        return "Non-bias detected"
    else:
        return "Unclear"

# Test classification
new_text = "Some potentially biased text."
print(classify_bias(new_text))


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import torch

# Load BERT model and tokenizer
# BERT tokenizer is loaded to tokenize input text, and model is loaded to use pre-trained BERT for sequence classification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # 2 labels: biased, non-biased

# Load dataset (you can use your dataset instead)
# Loading a pre-existing dataset called 'emotion' from Hugging Face
# Replace 'emotion' with your dataset for bias detection
dataset = load_dataset('emotion')

# Prepare the dataset with labels for bias (1 for bias, 0 for non-bias)
# Extracting texts and assigning labels manually based on a condition (e.g., keyword indicating bias)
train_texts = [example['text'] for example in dataset['train']]
train_labels = [1 if 'bias_keyword' in text.lower() else 0 for text in train_texts]  # Example labels, replace with real conditions

# Tokenize the data and add labels
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

# Create tokenized dataset
# Converting data to a Hugging Face Dataset format and setting format for PyTorch compatibility
tokenized_dataset = Dataset.from_dict({'text': train_texts, 'label': train_labels})
tokenized_dataset = tokenized_dataset.map(tokenize_function, batched=True)
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Split the dataset into training and evaluation
train_dataset = tokenized_dataset.select(range(int(0.8 * len(tokenized_dataset))))
eval_dataset = tokenized_dataset.select(range(int(0.8 * len(tokenized_dataset)), len(tokenized_dataset)))

# Training arguments
# Setting up training parameters such as batch size, number of epochs, logging frequency, etc.
training_args = TrainingArguments(
    output_dir="./results",  # Directory to save the training results
    evaluation_strategy="epoch",  # Evaluate the model at the end of each epoch
    per_device_train_batch_size=8,  # Batch size for training
    num_train_epochs=3,  # Number of epochs to train the model
    save_steps=500,  # Save the model every 500 steps
    logging_dir="./logs",  # Directory to save the logs
    logging_steps=10,  # Log every 10 steps
)

# Create Trainer instance
# Initializing the Trainer class with the model, training arguments, dataset, and tokenizer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=lambda data: {
        'input_ids': torch.stack([f['input_ids'] for f in data]),
        'attention_mask': torch.stack([f['attention_mask'] for f in data]),
        'labels': torch.tensor([f['label'] for f in data], dtype=torch.long)
    }
)

# Train the model
# Start training the model using the Trainer instance
trainer.train()

# Use the fine-tuned model to classify new texts
# Function to classify a given text as biased or non-biased using the fine-tuned model
def classify_bias(text):
    # Encode the input text to get input IDs for the model
    inputs = tokenizer(text, return_tensors="pt")
    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits).item()
    
    # Check the predicted class
    if predicted_class == 1:
        return "Bias detected"
    else:
        return "Non-bias detected"

# Test classification
# Testing the classification function with a sample text
new_text = "Some potentially biased text."
print(classify_bias(new_text))

# Note: To run this script, please install the required dependencies by running the following command:
# pip install transformers[torch] accelerate>=0.26.0


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import os
from datasets import load_dataset, Dataset
import torch

# Set environment variables for M1 GPU compatibility
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

# Load BERT model and tokenizer
# BERT tokenizer is loaded to tokenize input text, and model is loaded to use pre-trained BERT for sequence classification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # 2 labels: biased, non-biased

# Load dataset (you can use your dataset instead)
# Loading a pre-existing dataset called 'emotion' from Hugging Face
# Replace 'emotion' with your dataset for bias detection
dataset = load_dataset('emotion')

# Prepare the dataset with labels for bias (1 for bias, 0 for non-bias)
# Extracting texts and assigning labels manually based on a condition (e.g., keyword indicating bias)
train_texts = [example['text'] for example in dataset['train']]
train_labels = [1 if 'bias_keyword' in text.lower() else 0 for text in train_texts]  # Example labels, replace with real conditions

# Tokenize the data and add labels
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

# Create tokenized dataset
# Converting data to a Hugging Face Dataset format and setting format for PyTorch compatibility
tokenized_dataset = Dataset.from_dict({'text': train_texts, 'label': train_labels})
tokenized_dataset = tokenized_dataset.map(tokenize_function, batched=True)
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Split the dataset into training and evaluation
train_dataset = tokenized_dataset.select(range(int(0.8 * len(tokenized_dataset))))
eval_dataset = tokenized_dataset.select(range(int(0.8 * len(tokenized_dataset)), len(tokenized_dataset)))

# Training arguments
# Setting up training parameters such as batch size, number of epochs, logging frequency, etc.
training_args = TrainingArguments(
    output_dir="./results",  # Directory to save the training results
    evaluation_strategy="epoch",  # Evaluate the model at the end of each epoch
    per_device_train_batch_size=8,  # Batch size for training
    num_train_epochs=3,  # Number of epochs to train the model
    save_steps=500,  # Save the model every 500 steps
    logging_dir="./logs",  # Directory to save the logs
    logging_steps=10,  # Log every 10 steps
)

# Create Trainer instance
# Initializing the Trainer class with the model, training arguments, dataset, and tokenizer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=lambda data: {
        'input_ids': torch.stack([f['input_ids'] for f in data]),
        'attention_mask': torch.stack([f['attention_mask'] for f in data]),
        'labels': torch.tensor([f['label'] for f in data], dtype=torch.long)
    }
)

# Train the model
# Start training the model using the Trainer instance
trainer.train()

# Use the fine-tuned model to classify new texts
# Function to classify a given text as biased or non-biased using the fine-tuned model
def classify_bias(text):
    # Encode the input text to get input IDs for the model
    inputs = tokenizer(text, return_tensors="pt")
    # Get model predictions
    inputs = inputs.to('mps')  # Move inputs to M1 GPU
    model.to('mps')
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits).item()
    
    # Check the predicted class
    if predicted_class == 1:
        return "Bias detected"
    else:
        return "Non-bias detected"

# Test classification
# Testing the classification function with a sample text
new_text = "Some potentially biased text."
print(classify_bias(new_text))

# Note: To run this script, please install the required dependencies by running the following command:
# pip install transformers[torch] accelerate>=0.26.0
